## Params and loading packages

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']



In [2]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Modules.Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 16)


/home/isshamie/software/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


### Till here

### Merge based on tissue and type. And annotate

In [3]:
if not os.path.exists('Results'):
    os.mkdir('Results')
    
if not os.path.exists('Results/merged'):
    os.mkdir('Results/merged')

In [4]:
all_peak_merge_files = []

for curr_tissue in tissues:
    curr_raw = glob.glob(data_folder + curr_tissue + '/*/')
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if not curr_type == 'ATAC':
            count = 1
            for f in  glob.glob(j + '*f04_peaks/merge_bg_2.peak'):
                all_peak_merge_files.append(f)

output_file = 'Results/merged/all_peaks_merged.tsv'                
merge_peaks(all_peak_merge_files,output_file,dist='given')


output_anno_file = 'Results/merged/all_peaks_merged_anno.tsv' 
annotate_peaks(peak_file=output_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

mergePeaks -d given -venn Results/merged/all_peaks_merged.tsv_venn.tsv -matrix Results/merged/all_peaks_merged.tsv -strand /data/isshamie/TSS/Processed/BMDMwt/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Brain/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Brain/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Heart/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Kidney/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Kidney/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Liver/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Liver/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Lung/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Lung/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Muscle/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Pancreas/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Spleen/GRO/f04_peaks/merge_bg_2.peak /data

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


annotatePeaks.pl Results/merged/all_peaks_merged.tsv /data/isshamie/genome/picr.fa -gff /data/isshamie/genome/updated_final_sort.gff3 > Results/merged/all_peaks_merged_anno.tsv


### Merge based on any sample file (so replicates are treated separately)
#### And annotate

In [5]:
all_peak_merge_files = []

for curr_tissue in tissues:
    curr_raw = glob.glob(data_folder + curr_tissue + '/*/')
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if not curr_type == 'ATAC':
            count = 1
            for f in  glob.glob(j + '*f04_peaks/trim*.peak'):
                all_peak_merge_files.append(f)

output_file = 'Results/merged/duplicatesSeparate_peaks_merged.tsv'                
merge_peaks(all_peak_merge_files,output_file,dist='given')
output_anno_file = 'Results/merged/duplicateSeparate_peaks_merged_anno.tsv' 
annotate_peaks(peak_file=output_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

mergePeaks -d given -venn Results/merged/duplicatesSeparate_peaks_merged.tsv_venn.tsv -matrix Results/merged/duplicatesSeparate_peaks_merged.tsv -strand /data/isshamie/TSS/Processed/BMDMwt/GRO/f04_peaks/trim_CHBMDMwt_5GRO_JHS1033_SD_TAGCTT_S48_L003_R1_001.fastq_and_trim_CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Brain/GRO/f04_peaks/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq_and_trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Brain/mSTART/f04_peaks/trim_CHBrain_mSTART_JHS1057_SD_ATTCCT_S79_L004_R1_001.fastq_and_trim_CHBrain_mSTARTinput_JHS1078_SD_ATTCCT_S100_L005_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Brain/mSTART/f04_peaks/trim_CHBrain_mSTART_JHS1016_SD_GTAGAG_S28_L002_R1_001.fastq_and_trim_CHBrain_mSTARTinput_JHS1078_SD_ATTCCT_S100_L005_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Heart/mSTART/f04_peaks/trim_CHHeart_mSTART_JHS1056_SD_ATGAGC_S78_L004_R1_00

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


annotatePeaks.pl Results/merged/duplicatesSeparate_peaks_merged.tsv /data/isshamie/genome/picr.fa -gff /data/isshamie/genome/updated_final_sort.gff3 > Results/merged/duplicateSeparate_peaks_merged_anno.tsv


## Merge CHO all GRO and mSTART

In [10]:
all_peak_merge_files = []
curr_tissue ='CHO'
curr_raw = glob.glob(data_folder + curr_tissue + '/*/')
for j in curr_raw:
    curr_type = j.split('/')[-2]  
    if not curr_type == 'ATAC':
        count = 1
        for f in  glob.glob(j + '*f04_peaks/trim*.peak'):
            all_peak_merge_files.append(f)

output_file = 'Results/merged/CHO_samples_peaks_merged.tsv'                
merge_peaks(all_peak_merge_files,output_file,dist='given')
output_anno_file = 'Results/merged/CHO_samples_peaks_merged_anno.tsv' 
annotate_peaks(peak_file=output_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

mergePeaks -d given -venn Results/merged_samples/CHO_samples_peaks_merged.tsv_venn.tsv -matrix Results/merged_samples/CHO_samples_peaks_merged.tsv -strand /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD371_S10_R1_001_and_trim_CHO-GRO-SD369_S8_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD367_S6_R1_001_and_trim_CHO-GRO-SD370_S9_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD367_S6_R1_001_and_trim_CHO-GRO-SD369_S8_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD371_S10_R1_001_and_trim_CHO-GRO-SD370_S9_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/mSTART/f04_peaks/trim_CHO--mSTART-JHS823_S21_R1_001_and_trim_CHO--mSTART_input-JHS802_S11_R1_001_bg_2.peak > Results/merged_samples/CHO_samples_peaks_merged.tsv
annotatePeaks.pl Results/merged_samples/CHO_samples_peaks_merged.tsv /data/genome/hamster/picr/picr.fa -gff /data/genome/hamster/picr/updated_final_sort.gff3 > Results/m

## Use merged peaks to make histograms

In [1]:
anno_file = 'Results/merged/CHO_samples_peaks_merged_anno.tsv'
mRNA_peak_file= doc["mRNA_peak_file"] #"/data/isshamie/CH_tissue_TSS/mRNA.peak"
distance_from_promoter = 1000

out_folder = 'Results/merged'

In [ ]:
anno_peak(input_file,output_file)
peak_cov_hist(input_file,output_file,pc=0)
peak_cov_hist_mrna(input_file,output_file,pc=0)
peak_cov_hist_anno(input_file,output_file,pc=0)

## Covert to sequece

In [5]:
!pos2bed.pl Results/merged/all_peaks_merged_anno.tsv > Results/merged/all_peaks_merged.bed


	Converted 340547 peaks total



In [6]:
!pos2bed.pl Results/merged/duplicateSeparate_peaks_merged_anno.tsv > Results/merged/duplicateSeparate_peaks_merged_anno.bed


	Converted 470183 peaks total



In [14]:
f_in = 'Results/merged/duplicateSeparate_peaks_merged_anno.tsv'
ref_fa = '/data/isshamie/genome/picr.fa'
f_out = 'Results/merged/duplicateSeparate_peaks_merged_anno.fa'
cmd = 'homerTools extract {f_in} {ref_fa} -fa > {f_out}'.format(f_in=f_in,ref_fa=ref_fa,f_out=f_out)
!{cmd}


	Extracting sequences from file: /data/isshamie/genome/picr.fa
	Looking for peak sequences in a single file (/data/isshamie/genome/picr.fa)
	Extracting 18443 sequences from picr_0
	Extracting 14493 sequences from picr_1
	Extracting 15705 sequences from picr_2
	Extracting 12042 sequences from picr_3
	Extracting 10848 sequences from picr_4
	Extracting 9584 sequences from picr_5
	Extracting 7324 sequences from picr_6
	Extracting 12861 sequences from picr_8
	Extracting 7945 sequences from picr_9
	Extracting 7214 sequences from picr_10
	Extracting 6525 sequences from picr_11
	Extracting 3661 sequences from picr_7b
	Extracting 6327 sequences from picr_12
	Extracting 7128 sequences from picr_13
	Extracting 5878 sequences from picr_14
	Extracting 5086 sequences from picr_15
	Extracting 6151 sequences from picr_16
	Extracting 3343 sequences from picr_17
	Extracting 3745 sequences from picr_18
	Extracting 5325 sequences from picr_19
	Extracting 5976 sequences from picr_20
	Extracting 3803 seque

	Extracting 2 sequences from picr_436
	Extracting 4 sequences from picr_438
	Extracting 1 sequences from picr_441
	Extracting 1 sequences from picr_443
	Extracting 1 sequences from picr_445
	Extracting 1 sequences from picr_452
	Extracting 8 sequences from picr_454
	Extracting 1 sequences from picr_455
	Extracting 7 sequences from picr_456
	Extracting 5 sequences from picr_458
	Extracting 1 sequences from picr_459
	Extracting 20 sequences from picr_460
	Extracting 1 sequences from picr_461
	Extracting 2 sequences from picr_462
	Extracting 6 sequences from picr_463
	Extracting 4 sequences from picr_466
	Extracting 3 sequences from picr_469
	Extracting 1 sequences from picr_472
	Extracting 2 sequences from picr_474
	Extracting 8 sequences from picr_477
	Extracting 2 sequences from picr_478
	Extracting 1 sequences from picr_480
	Extracting 1 sequences from picr_481
	Extracting 3 sequences from picr_483
	Extracting 2 sequences from picr_484
	Extracting 1 sequences from picr_485
	Extracting

	Extracting 6 sequences from picr_1634
	Extracting 1 sequences from picr_1646
	Extracting 2 sequences from picr_1647
	Extracting 1 sequences from picr_1649
	Extracting 1 sequences from picr_1651
	Extracting 1 sequences from picr_1655
	Extracting 1 sequences from picr_1657
	Extracting 1 sequences from picr_1661
	Extracting 8 sequences from picr_1669
	Extracting 3 sequences from picr_1677
	Extracting 1 sequences from picr_1678
	Extracting 1 sequences from picr_1685
	Extracting 1 sequences from picr_1691
	Extracting 1 sequences from picr_1715
	Extracting 1 sequences from picr_1718
	Extracting 1 sequences from picr_1750
	Extracting 1 sequences from picr_1753
	Extracting 8 sequences from picr_1757
	Extracting 3 sequences from picr_1759
	Extracting 1 sequences from picr_1768
	Extracting 1 sequences from picr_1769
	Extracting 3 sequences from picr_1771
	Extracting 1 sequences from picr_1773
	Extracting 1 sequences from picr_1794
	Extracting 1 sequences from picr_1796
	Extracting 1 sequences f